In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
from matplotlib import pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import DataLoader
from torchvision import transforms

from datasets.flickr8k import Flickr8kDataset
from glove import embedding_matrix_creator
from metrics import *
from utils_torch import *

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
DATASET_BASE_PATH = 'data/flickr8k/'

In [5]:
train_set = Flickr8kDataset(dataset_base_path=DATASET_BASE_PATH, dist='train', device=device,
                            return_type='tensor',
                            load_img_to_memory=False)

vocab, word2idx, idx2word, max_len = vocab_set = train_set.get_vocab()

val_set = Flickr8kDataset(dataset_base_path=DATASET_BASE_PATH, dist='val', vocab_set=vocab_set, device=device,
                          return_type='corpus',
                          load_img_to_memory=False)
test_set = Flickr8kDataset(dataset_base_path=DATASET_BASE_PATH, dist='test', vocab_set=vocab_set, device=device,
                           return_type='corpus',
                           load_img_to_memory=False)
train_eval_set = Flickr8kDataset(dataset_base_path=DATASET_BASE_PATH, dist='train', vocab_set=vocab_set, device=device,
                                 return_type='corpus',
                                 load_img_to_memory=False)
                                 
with open('vocab_set.pkl', 'wb') as f:
    pickle.dump(train_set.get_vocab(), f)
len(train_set), len(val_set), len(test_set)

(30000, 1000, 1000)

In [6]:
vocab_size = len(vocab)
vocab_size, max_len

(7708, 40)

In [7]:
MODEL = "vgg16_monolstm"
EMBEDDING_DIM = 300
EMBEDDING = f"{EMBEDDING_DIM}"
ATTENTION_DIM = 256
DECODER_SIZE = 256
BATCH_SIZE = 10#128
HIDDEN_SIZE = 256
LR = 5e-4
MODEL_NAME = f'saved_models/{MODEL}_b{BATCH_SIZE}_emd{EMBEDDING}'
NUM_EPOCHS = 2
SAVE_FREQ = 10
LOG_INTERVAL = 25 * (256 // BATCH_SIZE)

In [8]:
embedding_matrix = embedding_matrix_creator(embedding_dim=EMBEDDING_DIM, word2idx=word2idx)
embedding_matrix.shape

100%|██████████| 7708/7708 [00:00<00:00, 528798.71it/s]


(7708, 300)

In [9]:
def train_model(train_loader, model, loss_fn, optimizer, vocab_size, acc_fn, desc=''):
    running_acc = 0.0
    running_loss = 0.0
    model.train()
    t = tqdm(iter(train_loader), desc=f'{desc}')
    for batch_idx, batch in enumerate(t):
        images, captions, lengths = batch

        optimizer.zero_grad()

        scores, caps_sorted, decode_lengths, alphas, sort_ind = model(images, captions, lengths)

        # Since decoding starts with <start>, the targets are all words after <start>, up to <end>
        targets = caps_sorted[:, 1:]
        print(targets.shape)

        # Remove timesteps that we didn't decode at, or are pads
        # pack_padded_sequence is an easy trick to do this
        scores = pack_padded_sequence(scores, decode_lengths, batch_first=True)
        targets = pack_padded_sequence(targets, decode_lengths, batch_first=True)
        print(type(scores))

        loss = loss_fn(scores, targets)
        loss.backward()
        optimizer.step()

        running_acc += (torch.argmax(scores, dim=1) == targets).sum().float().item() / targets.size(0)
        running_loss += loss.item()
        t.set_postfix({'loss': running_loss / (batch_idx + 1),
                       'acc': running_acc / (batch_idx + 1),
                       }, refresh=True)
        if (batch_idx + 1) % LOG_INTERVAL == 0:
            print(f'{desc} {batch_idx + 1}/{len(train_loader)} '
                  f'train_loss: {running_loss / (batch_idx + 1):.4f} '
                  f'train_acc: {running_acc / (batch_idx + 1):.4f}')

    return running_loss / len(train_loader)


def evaluate_model(data_loader, model, loss_fn, vocab_size, bleu_score_fn, tensor_to_word_fn, desc=''):
    running_bleu = [0.0] * 5
    model.eval()
    t = tqdm(iter(data_loader), desc=f'{desc}')
    for batch_idx, batch in enumerate(t):
        images, captions, lengths = batch
        outputs = tensor_to_word_fn(model.sample(images, startseq_idx=word2idx['<start>']).cpu().numpy())

        for i in (1, 2, 3, 4):
            running_bleu[i] += bleu_score_fn(reference_corpus=captions, candidate_corpus=outputs, n=i)
        t.set_postfix({
            'bleu1': running_bleu[1] / (batch_idx + 1),
            'bleu4': running_bleu[4] / (batch_idx + 1),
        }, refresh=True)
    for i in (1, 2, 3, 4):
        running_bleu[i] /= len(data_loader)
    return running_bleu

In [11]:
from models.torch.vgg16_monolstm import Captioner

final_model = Captioner(
    embed_size=EMBEDDING_DIM, hidden_size=HIDDEN_SIZE,
    num_layers=1, vocab_size=vocab_size,
    embedding_matrix=embedding_matrix, train_embd=True
).to(device)


loss_fn = torch.nn.CrossEntropyLoss(ignore_index=train_set.pad_value).to(device)
acc_fn = accuracy_fn(ignore_value=train_set.pad_value)
sentence_bleu_score_fn = bleu_score_fn(4, 'sentence')
corpus_bleu_score_fn = bleu_score_fn(4, 'corpus')
tensor_to_word_fn = words_from_tensors_fn(idx2word=idx2word)

params = final_model.parameters()

optimizer = torch.optim.Adam(params=params, lr=LR)

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [12]:
train_transformations = transforms.Compose([
    transforms.Resize(256),  # smaller edge of image resized to 256
    transforms.RandomCrop(256),  # get 256x256 crop from random location
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),  # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),  # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))
])
eval_transformations = transforms.Compose([
    transforms.Resize(256),  # smaller edge of image resized to 256
    transforms.CenterCrop(256),  # get 256x256 crop from random location
    transforms.ToTensor(),  # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),  # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))
])

train_set.transformations = train_transformations
val_set.transformations = eval_transformations
test_set.transformations = eval_transformations
train_eval_set.transformations = eval_transformations

In [13]:
eval_collate_fn = lambda batch: (torch.stack([x[0] for x in batch]), [x[1] for x in batch], [x[2] for x in batch])
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, sampler=None, pin_memory=False)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, sampler=None, pin_memory=False,
                        collate_fn=eval_collate_fn)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, sampler=None, pin_memory=False,
                         collate_fn=eval_collate_fn)
train_eval_loader = DataLoader(train_eval_set, batch_size=BATCH_SIZE, shuffle=False, sampler=None, pin_memory=False,
                               collate_fn=eval_collate_fn)

In [14]:
train_loss_min = 100
val_bleu4_max = 0.0
for epoch in range(NUM_EPOCHS):
    train_loss = train_model(desc=f'Epoch {epoch + 1}/{NUM_EPOCHS}', model=final_model,
                             optimizer=optimizer, loss_fn=loss_fn, acc_fn=acc_fn,
                             train_loader=train_loader, vocab_size=vocab_size)
    with torch.no_grad():
        train_bleu = evaluate_model(desc=f'\tTrain Bleu Score: ', model=final_model,
                                    loss_fn=loss_fn, bleu_score_fn=corpus_bleu_score_fn,
                                    tensor_to_word_fn=tensor_to_word_fn,
                                    data_loader=train_eval_loader, vocab_size=vocab_size)
        val_bleu = evaluate_model(desc=f'Validation Bleu Score: ', model=final_model,
                                  loss_fn=loss_fn, bleu_score_fn=corpus_bleu_score_fn,
                                  tensor_to_word_fn=tensor_to_word_fn,
                                  data_loader=val_loader, vocab_size=vocab_size)
        print(f'Epoch {epoch + 1}/{NUM_EPOCHS}',
              ''.join([f'train_bleu{i}: {train_bleu[i]:.4f} ' for i in (1, 4)]),
              ''.join([f'val_bleu{i}: {val_bleu[i]:.4f} ' for i in (1, 4)]),
              )
        #wandb.log({f'val_bleu{i}': val_bleu[i] for i in (1, 2, 3, 4)})
        #wandb.log({'val_bleu': val_bleu[4]})
        state = {
            'epoch': epoch + 1,
            'state_dict': final_model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'train_loss_latest': train_loss,
            'val_bleu4_latest': val_bleu[4],
            'train_loss_min': min(train_loss, train_loss_min),
            'val_bleu4_max': max(val_bleu[4], val_bleu4_max),
            'train_bleus': train_bleu,
            'val_bleus': val_bleu,
        }
        torch.save(state, f'{MODEL_NAME}_latest.pt')
        #wandb.save(f'{MODEL_NAME}_latest.pt')
        if train_loss < train_loss_min:
            train_loss_min = train_loss
            torch.save(state, f'{MODEL_NAME}''_best_train.pt')
            #wandb.save(f'{MODEL_NAME}''_best_train.pt')
        if val_bleu[4] > val_bleu4_max:
            val_bleu4_max = val_bleu[4]
            torch.save(state, f'{MODEL_NAME}''_best_val.pt')
            #wandb.save(f'{MODEL_NAME}''_best_val.pt')

torch.save(state, f'{MODEL_NAME}_ep{NUM_EPOCHS:02d}_weights.pt')
#wandb.save(f'{MODEL_NAME}_ep{NUM_EPOCHS:02d}_weights.pt')
final_model.eval()

Epoch 1/2:   0%|          | 0/3000 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Epoch 1/2:   0%|          | 0/3000 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 5)

In [ ]:
model = final_model

In [ ]:
t_i = 1003
dset = train_set
im, cp, _ = dset[t_i]
print(''.join([idx2word[idx.item()] + ' ' for idx in model.sample(im.unsqueeze(0), word2idx['<start>'])[0]]))
print(dset.get_image_captions(t_i)[1])

plt.imshow(dset[t_i][0].detach().cpu().permute(1, 2, 0), interpolation="bicubic")

In [ ]:
t_i = 500
dset = val_set
im, cp, _ = dset[t_i]
print(''.join([idx2word[idx.item()] + ' ' for idx in model.sample(im.unsqueeze(0), word2idx['<start>'])[0]]))
print(cp)

plt.imshow(dset[t_i][0].detach().cpu().permute(1, 2, 0), interpolation="bicubic")

In [ ]:
t_i = 500
dset = test_set
im, cp, _ = dset[t_i]
print(''.join([idx2word[idx.item()] + ' ' for idx in model.sample(im.unsqueeze(0), word2idx['<start>'])[0]]))
print(cp)

plt.imshow(dset[t_i][0].detach().cpu().permute(1, 2, 0), interpolation="bicubic")

In [ ]:
with torch.no_grad():
    model.eval()
    train_bleu = evaluate_model(desc=f'Train: ', model=final_model,
                                loss_fn=loss_fn, bleu_score_fn=corpus_bleu_score_fn,
                                tensor_to_word_fn=tensor_to_word_fn,
                                data_loader=train_eval_loader, vocab_size=vocab_size)
    val_bleu = evaluate_model(desc=f'Val: ', model=final_model,
                              loss_fn=loss_fn, bleu_score_fn=corpus_bleu_score_fn,
                              tensor_to_word_fn=tensor_to_word_fn,
                              data_loader=val_loader, vocab_size=vocab_size)
    test_bleu = evaluate_model(desc=f'Test: ', model=final_model,
                               loss_fn=loss_fn, bleu_score_fn=corpus_bleu_score_fn,
                               tensor_to_word_fn=tensor_to_word_fn,
                               data_loader=test_loader, vocab_size=vocab_size)
    for setname, result in zip(('train', 'val', 'test'), (train_bleu, val_bleu, test_bleu)):
        print(setname, end=' ')
        for ngram in (1, 2, 3, 4):
            print(f'Bleu-{ngram}: {result[ngram]}', end=' ')
            wandb.run.summary[f"{setname}_bleu{ngram}"] = result[ngram]
        print()